## Q26 双线性插值 (Bilinear Interpolation)

-----
[参考](https://www.cnblogs.com/sdxk/p/4056223.html)
------

双线性插值考察4邻域的像素点，根据距离设置权值。虽然计算量增大使得处理时间变长，但是可以有效抑制画质劣化





In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [4]:
img = cv2.imread("gg.jpg")

(H, W, C) = img.shape
print("H: ", H, "W:", W)

ratio = 1.5

Height = int(H * ratio)
Width = int(W * ratio)
print("after H: ", Height, "after W:", Width)

img_out = np.zeros((Height, Width, C), dtype = np.int)

for height in range(Height):
    for width in range(Width):
        
        h = int(height / ratio)
        w = int(width / ratio)
        # 边界处理
        if w > W - 2:
            w = W - 2
        if h > H - 2:
            h = H - 2
        
#         fR1 = (width / ratio - w) * img[h , w+1] + (w + 1 - (width / ratio)) * img[h, w]
#         fR2 = (width / ratio - w) * img[h + 1 , w+1] + (w + 1 - (width / ratio)) * img[h + 1, w]
#         img_out[height, width] = (height / ratio - h) * fR2 + (h + 1 - (height / ratio)) * fR1
        
        dh = h - int(height / ratio)
        dw = w - int(width / ratio)
        img_out[height, width] = dh* dw * img[h + 1, w + 1] + (1-dh) * dw * img[h, w + 1] + \
                                dh * (1- dw) * img[h+1, w] + (1- dh) * (1- dw) * img[h + 1, w + 1]

img_out = img_out.astype(np.uint8)

# img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# img_out = cv2.cvtColor(img_out, cv2.COLOR_BGR2RGB)
            
cv2.imshow("img", img)
cv2.imshow("img_out", img_out)
cv2.waitKey()


H:  128 W: 128
after H:  192 after W: 192


-1

## 双三次插值Bicubic Interpolation

[参考](https://blog.csdn.net/linqianbi/article/details/78594019)

In [1]:
img = cv2.imread("gg.jpg")

(H, W, C) = img.shape

ratio = 1.5

Height = int(H * ratio)
Width = int(W * ratio)

img_out = np.zeros((Height, Width, C), dtype=np.int)
d = np.zeros((4, 4), dtype= np.int)

for height in range(Height):
    for width in range(Width):
        h = int(height / ratio)
        w = int(width / ratio)
        # 边界处理
        if w > W - 2:
            w = W - 2
        if h > H - 2:
            h = H - 2
        
        
        
        


SyntaxError: unexpected EOF while parsing (<ipython-input-1-204cc3444621>, line 12)